# Predição de Falhas em Equipamentos Industriais com XGBoost

Este notebook detalha a construção de um modelo de **manutenção preditiva** utilizando o algoritmo **XGBoost**. O objetivo principal é prever falhas iminentes em um equipamento industrial (ITU_693) com **30 minutos de antecedência**. 

A metodologia se baseia na seguinte lógica:
1.  **Definição de Falha**: Uma falha é caracterizada por uma queda brusca e significativa no RPM do motor (superior a 70%).
2.  **Janela de Pré-Falha**: O período de 30 minutos que antecede essa queda é rotulado como 'pré-falha'.
3.  **Treinamento do Modelo**: O modelo XGBoost é treinado para reconhecer os padrões dos dados de sensores que ocorrem consistentemente nesta janela de pré-falha.
4.  **Otimização**: O modelo é otimizado com `GridSearchCV` para maximizar a capacidade de detectar falhas reais (métrica de `recall` ou `f1-score`), que é o objetivo crítico do negócio.

O resultado é um sistema capaz de emitir alertas proativos, permitindo que a equipe de manutenção atue antes da ocorrência de uma parada não programada, reduzindo custos e aumentando a disponibilidade do equipamento.

## Configuração do Ambiente e Bibliotecas

A primeira etapa consiste em importar as bibliotecas essenciais para manipulação de dados (`pandas`, `numpy`), que são a base para qualquer projeto de Data Science.

In [1]:
import pandas as pd
import numpy as np

## Carga e Preparação Inicial dos Dados

Carregamos o dataset já processado (`itu_693_processed.csv`). A etapa mais crucial aqui é a conversão da coluna `timestamp` para o formato `datetime` do pandas. Isso é fundamental para realizar operações temporais, como definir a janela de pré-falha, e para garantir a divisão cronológica correta dos dados.

In [2]:
df = pd.read_csv("../../data_cleaning_pipeline/itu_693_processed.csv")
df['timestamp'] = pd.to_datetime(df['timestamp']) 

O comando `df.info()` é uma verificação rápida para garantir que os dados foram carregados corretamente, que não há valores nulos inesperados e que os tipos de dados (especialmente o `timestamp`) estão corretos antes de prosseguir.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505922 entries, 0 to 505921
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   timestamp                      505922 non-null  datetime64[ns, UTC]
 1   Auto                           505922 non-null  float64            
 2   Bat_V                          505922 non-null  float64            
 3   Char_V                         505922 non-null  float64            
 4   Cool_T                         505922 non-null  float64            
 5   Eng_RPM                        505922 non-null  float64            
 6   Fuel_Con                       505922 non-null  float64            
 7   Fuel_L                         505922 non-null  float64            
 8   Man                            505922 non-null  float64            
 9   Oil_L                          505922 non-null  float64            
 10  Oil_P   

## Definição da Falha e Rotulagem dos Dados (Target Engineering)

Esta é a etapa mais crítica do projeto, onde a regra de negócio é traduzida em lógica de programação para criar nossa variável-alvo (`failure_label`).

1.  **Definição do Evento de Falha**: Uma falha é definida como uma queda de RPM de **70% ou mais** (`RPM_DROP_THRESHOLD_PERCENTAGE`). Este limiar deve ser definido com base no conhecimento do especialista do equipamento. Um valor negativo indica uma queda.
2.  **Janela de Pré-Falha**: Definimos uma janela de **30 minutos** (`PRE_FAILURE_WINDOW_MINUTES`) antes do evento de falha. O objetivo do modelo é aprender a identificar os padrões de dados *dentro* desta janela.
3.  **Criação da Label**: Inicializamos a coluna `failure_label` com `0` (estado normal). Em seguida, para cada evento de falha encontrado, iteramos e marcamos todas as amostras dentro da janela de 30 minutos que o antecede com o valor `1` (pré-falha).

Este processo transforma um problema de detecção de anomalias em um problema de **classificação binária supervisionada**, que é mais fácil de modelar e avaliar.

In [4]:
# 1. Limiar da Queda de RPM (em porcentagem)
# Uma queda de 70% ou mais em um único intervalo de tempo (12s no seu caso).
# Usamos um valor negativo pois representa uma queda.
RPM_DROP_THRESHOLD_PERCENTAGE = -70.0
PRE_FAILURE_WINDOW_MINUTES = 30

# Inicializa a coluna alvo com 0 (operação normal)
df['failure_label'] = 0

# Identifica os pontos exatos onde a falha (queda de RPM) ocorre
failure_events = df[df['Eng_RPM_variation_percentage'] <= RPM_DROP_THRESHOLD_PERCENTAGE]
print(f"\nForam encontrados {len(failure_events)} eventos de queda brusca de RPM.")


Foram encontrados 740 eventos de queda brusca de RPM.


In [5]:
# Itera sobre cada evento de falha para rotular a janela de pré-falha correspondente
if not failure_events.empty:
    for index, event_row in failure_events.iterrows():
        failure_timestamp = event_row['timestamp']
        
        # Calcula o início da janela de pré-falha (30 minutos antes da falha)
        window_start_timestamp = failure_timestamp - pd.Timedelta(minutes=PRE_FAILURE_WINDOW_MINUTES)
        
        # Cria uma máscara booleana para identificar todas as linhas dentro da janela de pré-falha
        window_mask = (df['timestamp'] >= window_start_timestamp) & (df['timestamp'] < failure_timestamp)
        
        # Atribui o rótulo 1 (pré-falha) para todas as amostras dentro dessa janela
        df.loc[window_mask, 'failure_label'] = 1
        
    print("\nRotulagem das janelas de pré-falha concluída.")
else:
    print("\nNenhum evento de falha encontrado com os parâmetros atuais.")



Rotulagem das janelas de pré-falha concluída.


## Análise da Distribuição das Classes

Verificamos a contagem de cada classe (`0` para Normal, `1` para Pré-Falha). Como esperado em problemas de manutenção preditiva, o dataset é **altamente desbalanceado**: há muito mais dados de operação normal do que de pré-falha. É crucial estar ciente disso, pois afeta a escolha da métrica de avaliação e a configuração do modelo.

In [6]:
print("\nDistribuição da nova coluna 'failure_label':")
print(df['failure_label'].value_counts())


Distribuição da nova coluna 'failure_label':
failure_label
0    407175
1     98747
Name: count, dtype: int64


## Preparação para a Modelagem

Importamos as bibliotecas necessárias para a modelagem (`xgboost`) e avaliação (`sklearn.metrics`).

In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

### Separação de Features (X) e Alvo (y)

Separamos o dataset em duas partes:
- **`X` (Features)**: Todas as colunas que serão usadas como preditores. Excluímos a própria label (`failure_label`) e colunas não numéricas ou identificadoras (`timestamp`, `motor_pump`) que não devem ser usadas diretamente no treinamento.
- **`y` (Alvo/Target)**: A coluna que queremos prever, ou seja, `failure_label`.

In [21]:
y = df['failure_label']
X = df.drop(columns=['failure_label', 'timestamp', 'motor_pump'])

In [22]:
print("Features (X) e Alvo (y) definidos.")
print(f"Número de features: {X.shape[1]}")
print(f"Tamanho do dataset: {X.shape[0]} amostras")

Features (X) e Alvo (y) definidos.
Número de features: 39
Tamanho do dataset: 505922 amostras


### Divisão Cronológica dos Dados (Treino e Teste)

Para dados de séries temporais, uma divisão aleatória (`train_test_split`) é incorreta e leva a *data leakage* (vazamento de dados), pois o modelo aprenderia com dados futuros para prever o passado. A abordagem correta é a **divisão cronológica**: usamos os primeiros 80% dos dados para treinar o modelo e os 20% mais recentes para testá-lo. Isso simula um cenário real, onde usamos o histórico para prever o futuro.

In [23]:
split_point = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_point], X.iloc[split_point:]
y_train, y_test = y.iloc[:split_point], y.iloc[split_point:]

print("Dados divididos em treino e teste de forma cronológica:")
print(f" - Treino: {len(X_train)} amostras")
print(f" - Teste:  {len(X_test)} amostras")

Dados divididos em treino e teste de forma cronológica:
 - Treino: 404737 amostras
 - Teste:  101185 amostras


In [ ]:
from imblearn.combine import SMOTETomek
from collections import Counter

# Antes do balanceamento
print("Distribuição original:", Counter(y_train))

# Instancia o SMOTE Tomek
smote_tomek = SMOTETomek(random_state=42)

# Aplica no conjunto de treino (NUNCA no teste!)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)

# Depois do balanceamento
print("Distribuição após SMOTE Tomek:", Counter(y_resampled))


Distribuição original: Counter({0: 327643, 1: 77094})
Distribuição após SMOTE Tomek: Counter({1: 327424, 0: 327424})


### Tratamento do Desbalanceamento de Classes

Como vimos, a classe 'pré-falha' é rara. Se não fizermos nada, o modelo pode simplesmente aprender a prever 'normal' o tempo todo e ainda assim ter uma alta acurácia. Para combater isso, calculamos o `scale_pos_weight`. Este parâmetro do XGBoost informa ao modelo para dar mais importância (peso) aos erros cometidos na classe minoritária (pré-falha). O valor é calculado como `(número de amostras normais) / (número de amostras de pré-falha)`.

In [24]:
count_normal = y_train.value_counts()[0]
count_pre_failure = y_train.value_counts()[1]

scale_pos_weight = count_normal / count_pre_failure if count_pre_failure != 0 else 1
print(f"O parâmetro 'scale_pos_weight' é: {scale_pos_weight:.2f}")

O parâmetro 'scale_pos_weight' é: 4.25


## Treinamento do Modelo XGBoost (Baseline)

**Racional**: Instanciamos e treinamos um primeiro modelo XGBoost com parâmetros iniciais razoáveis. Este servirá como nossa linha de base (baseline) para comparação após a otimização.

- `objective='binary:logistic'`: Define o problema como classificação binária.
- `eval_metric='logloss'`: Métrica usada para avaliar o desempenho durante o treinamento.
- `scale_pos_weight`: Parâmetro crucial que calculamos para lidar com o desbalanceamento.
- `n_estimators`, `max_depth`, `learning_rate`: Hiperparâmetros que controlam a complexidade e o processo de aprendizado do modelo. Eles serão otimizados na próxima etapa.

In [25]:
print("\nIniciando o treinamento do modelo XGBoost...")

# Instanciando o modelo com os parâmetros
# 'objective='binary:logistic'' é para classificação binária
# 'eval_metric='logloss'' é uma métrica de erro comum
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight, # Usando o peso que calculamos
    n_estimators=200,                  # Número de "árvores" no modelo. Comece com um valor moderado.
    max_depth=6,                       # Profundidade máxima de cada árvore. Ajuda a evitar overfitting.
    learning_rate=0.1,                 # Taxa de aprendizado.
    use_label_encoder=False,           # Para evitar um aviso de depreciação
    random_state=42                    # Para reprodutibilidade
)


Iniciando o treinamento do modelo XGBoost...


In [50]:
# model.fit(X_train, y_train)
model.fit(X_resampled, y_resampled)

/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:38:30] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


## Avaliação do Modelo Baseline

Avaliamos o desempenho do modelo inicial no conjunto de teste, que ele nunca viu antes.

- **Matriz de Confusão**: Mostra os acertos e erros. O mais importante é olharmos para os **Verdadeiros Positivos** (falhas que previmos corretamente) e **Falsos Negativos** (falhas que não conseguimos prever - o pior erro para o negócio).
- **Relatório de Classificação**: Fornece métricas detalhadas. Para a classe 'Pré-Falha (1)', o **`recall`** é a métrica mais importante, pois indica a porcentagem de falhas reais que nosso modelo conseguiu capturar.
- **ROC AUC Score**: Mede a capacidade geral do modelo de distinguir entre as classes normal e de pré-falha. Um valor mais próximo de 1.0 é melhor.

In [47]:
y_pred = model.predict(X_test)
print("\n--- AVALIAÇÃO DO MODELO ---")

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix: {cm}")


y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC AUC Score: {auc}")


--- AVALIAÇÃO DO MODELO ---

Matriz de Confusão:
Confusion Matrix: [[75586  3946]
 [ 9161 12492]]

ROC AUC Score: 0.9354224232263986


## Otimização de Hiperparâmetros com GridSearchCV

O desempenho do modelo baseline pode ser melhorado ajustando seus hiperparâmetros. Fazer isso manualmente é tedioso e ineficiente. O `GridSearchCV` automatiza esse processo: ele testa exaustivamente todas as combinações de parâmetros que definimos (`param_grid`) e, por meio de validação cruzada (`cv=3`), encontra a combinação que maximiza a métrica de desempenho escolhida.

- **`scoring='f1'`**: Escolhemos o F1-Score como a métrica a ser otimizada. É uma boa escolha para datasets desbalanceados, pois busca um equilíbrio entre `precision` (evitar falsos alarmes) e `recall` (capturar falhas reais). Maximizar o `recall` diretamente também seria uma opção válida, dependendo do custo de um falso alarme versus o custo de uma falha não detectada.
- **`n_jobs=-1`**: Usa todos os núcleos de CPU disponíveis para acelerar o processo, que pode ser demorado.

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, recall_score

# Definindo os hiperparâmetros para testar
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

# Instanciando o classificador
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    use_label_encoder=False,
    random_state=42
)

# GridSearchCV com foco em maximizar o f1-score
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='f1',  # ou 'recall' se quiser maximizar a detecção a todo custo
    cv=3,              # validação cruzada com 3 folds
    verbose=1,
    n_jobs=-1
)

print("Iniciando GridSearchCV...")
grid_search.fit(X_train, y_train)

# Melhor modelo encontrado pelo GridSearch
best_model = grid_search.best_estimator_
print("\nMelhores parâmetros encontrados:")
print(grid_search.best_params_)

# Avaliação do melhor modelo no conjunto de teste
y_pred_best = best_model.predict(X_test)

print("\n--- AVALIAÇÃO DO MODELO APÓS GRIDSEARCH ---")
print("\nMatriz de Confusão:")
cm_best = confusion_matrix(y_test, y_pred_best)
print(f"Confusion Matrix: {cm_best}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_best, target_names=['Normal (0)', 'Pré-Falha (1)']))

# Probabilidades e AUC
y_pred_proba_best = best_model.predict_proba(X_test)[:, 1]
auc_best = roc_auc_score(y_test, y_pred_proba_best)
print(f"\nROC AUC Score: {auc_best}")

Iniciando GridSearchCV...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:39:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:39:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:39:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [14:39:05] WARNING: /workspace/src/learner.


Melhores parâmetros encontrados:
{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1.0}

--- AVALIAÇÃO DO MODELO APÓS GRIDSEARCH ---

Matriz de Confusão:
Confusion Matrix: [[73527  6005]
 [ 7537 14116]]

Relatório de Classificação:
               precision    recall  f1-score   support

   Normal (0)       0.91      0.92      0.92     79532
Pré-Falha (1)       0.70      0.65      0.68     21653

     accuracy                           0.87    101185
    macro avg       0.80      0.79      0.80    101185
 weighted avg       0.86      0.87      0.86    101185


ROC AUC Score: 0.9385477704828176


Análise de Importância das Features

Após treinar o modelo final, é fundamental entender *quais* variáveis (sensores) ele considera mais importantes para fazer as previsões. O XGBoost possui um atributo `feature_importances_` que nos dá essa informação.

**Aplicação Prática**:
- **Priorização**: Ajuda a equipe de manutenção a focar nos sensores mais críticos.
- **Simplificação**: Se algumas features tiverem importância quase nula, podem ser removidas em futuras versões do modelo para torná-lo mais rápido e simples.
- **Insights**: Pode revelar relações inesperadas entre variáveis e falhas, gerando conhecimento valioso sobre o equipamento.

In [29]:
# Extrai a importância das features do melhor modelo treinado
feature_importance = best_model.feature_importances_
feature_names = X.columns

# Cria um DataFrame para visualizar e ordenar as features mais importantes
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\n--- TOP 10 FEATURES MAIS IMPORTANTES ---")
print(importance_df.head(10).to_string())


--- TOP 10 FEATURES MAIS IMPORTANTES ---
     feature  importance
4    Eng_RPM    0.578312
1      Bat_V    0.131953
10  Recalque    0.119059
0       Auto    0.039677
5   Fuel_Con    0.028050
9      Oil_P    0.020072
7        Man    0.012747
12      Stop    0.011608
11  Starts_N    0.008688
3     Cool_T    0.008171


## Salvamento do Modelo Final

O objeto `best_model` contém todo o aprendizado do processo de treinamento e otimização. Nós o salvamos em um arquivo (usando `pickle`) para que possa ser carregado posteriormente em outro ambiente (um script de produção, uma API) para fazer previsões em dados novos, sem a necessidade de retreinar tudo do zero. Este é um passo essencial para o deploy do modelo.

In [30]:
import pickle

# Salva o melhor modelo encontrado pelo GridSearchCV em um arquivo .pkl
with open("xgb_best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("\nModelo final 'xgb_best_model.pkl' salvo com sucesso.")


Modelo final 'xgb_best_model.pkl' salvo com sucesso.


## Documentação Final e Conclusões

### Resumo

Este projeto desenvolveu com sucesso um sistema de predição de falhas para o equipamento ITU_693 utilizando XGBoost. O modelo foi treinado com dados históricos e otimizado através de Grid Search com validação cruzada para prever falhas com 30 minutos de antecedência.

**Principais Conquistas**:
- Sistema capaz de prever falhas com **30 minutos** de antecedência.
- Modelo otimizado para maximizar a **detecção de falhas** (recall/f1-score).
- Metodologia com **divisão cronológica** e tratamento de **classes desbalanceadas**.
- Modelo final salvo e pronto para ser implementado em um ambiente de produção.

### Métricas Finais do Sistema

**Performance do Modelo (no conjunto de teste)**:
- **Recall (Taxa de Detecção de Falhas)**: 65% da classe Pré-Falha.
- **Precision (Assertividade dos Alertas)**: 70% da classe Pré-Falha.
- **F1-Score (Balanço P/R)**: 68% da classe Pré-Falha.
- **ROC/AUC**: 94% da classe Pré-Falha

### Validação e Testes Realizados

**Metodologia de Validação**:
- Divisão temporal dos dados (80% treino, 20% teste).
- Validação cruzada (3-fold) na otimização de hiperparâmetros.
- Teste final em dados futuros (não vistos no treino).
- Análise de importância das features para interpretabilidade.

**Robustez do Modelo**:
- Tratamento de classes desbalanceadas com `scale_pos_weight`.
- Prevenção de overfitting via validação cruzada e parâmetros do XGBoost (e.g., `max_depth`).

### Recomendações Críticas

**Para Implementação Imediata**:
1.  **Deploy em Piloto**: Implementar o modelo em um ambiente de sombra/teste, alimentando-o com dados em tempo real para monitorar suas previsões sem gerar alertas operacionais.
2.  **Calibração do Threshold**: O threshold padrão de decisão é 0.5. Analisar a curva ROC para ajustar este valor pode otimizar o balanço entre falsos positivos e falsos negativos de acordo com a necessidade do negócio.
3.  **Treinamento da Equipe**: Capacitar a equipe de manutenção para interpretar os alertas do sistema e seguir os novos procedimentos.

**Para Sustentabilidade**:
1.  **Monitoramento Contínuo**: Implementar monitoramento de *data drift* para detectar quando os padrões dos dados mudam e o modelo começa a perder performance.
2.  **Retreinamento Periódico**: Agendar retreinamento do modelo (e.g., trimestralmente) com novos dados para garantir que ele se mantenha atualizado com o comportamento do equipamento.
3.  **Expansão**: Replicar a metodologia para outros equipamentos.